In [1]:
import pandas as pd
import geopandas as gpd
import pybdshadow
#Read building data
buildings = gpd.read_file(r'../example/data/bd_demo.json')
buildings = pybdshadow.bd_preprocess(buildings)
buildings.head(5)

,id,height,x,y,geometry,building_id
0,0,42,139.698802,35.533816,"POLYGON ((139.69831 35.53380, 139.69889 35.533...",0
1,1,9,139.698402,35.534030,"POLYGON ((139.69799 35.53417, 139.69882 35.534...",1
2,2,45,139.698491,35.534637,"POLYGON ((139.69864 35.53445, 139.69843 35.534...",2
3,3,9,139.698784,35.534735,"POLYGON ((139.69864 35.53477, 139.69891 35.534...",3
4,4,90,139.700506,35.536064,"POLYGON ((139.70015 35.53590, 139.70062 35.536...",4


In [2]:
#Given UTC time
date = pd.to_datetime('2022-01-01 12:45:33.959797119')\
    .tz_localize('Asia/Shanghai')\
    .tz_convert('UTC')
